<a href="https://colab.research.google.com/github/ThazSobral/tcc/blob/main/entropy_technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Técnica de entropia.

by: Sidgley Camargo

## Ajuste

Montagem do drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Bibliotecas

Importação de bibliotecas

In [ ]:
import pandas as pd

## Dataframe

Importação dos dados

In [ ]:
df = pd.read_csv('/content/drive/My Drive/TCC_Thalles Sobral/2-dados/germany/use-data/adjusted.csv')

Verificação dos dados

In [ ]:
df.head()

,Unnamed: 0,hashtags,tweet,created_at,X,Y,distance [m],sev08,sev09,sev10,flood,relevance,theme
0,0,NaN,irgendjemand hat mir gestern gesagt das er mic...,2013-06-08 13:33:07+02,"429,79","5,915,818",122596,-24,-67,-127,0,0,0
1,1,NaN,"@ho1ger das gleiche bild übrigens im office, w...",2013-06-08 13:33:09+02,"708,059","5,342,364",66842,1,-79,-114,0,0,0
2,2,NaN,@sensatzionell bin selbst halber berliner. wen...,2013-06-08 13:33:10+02,"797,734","5,829,275",67270,-8,-10,-20,0,0,0
3,3,NaN,tired bro??!!! djeeerome @ ice 672 http://t.co...,2013-06-08 13:33:14+02,"522,889","5,881,956",33741,-24,-67,-127,0,0,0
4,4,NaN,und tschüss! (@ köln bonn airport (cgn) w/ 8 o...,2013-06-08 13:33:14+02,"367,861","5,638,052",71798,-54,-105,-161,0,0,0


## Pré-processamento

In [ ]:
df.count

<bound method DataFrame.count of        Unnamed: 0 hashtags  ... relevance theme
0               0      NaN  ...         0     0
1               1      NaN  ...         0     0
2               2      NaN  ...         0     0
3               3      NaN  ...         0     0
4               4      NaN  ...         0     0
...           ...      ...  ...       ...   ...
60519       60519     gosh  ...         0     0
60520       60520      NaN  ...         0     0
60521       60521      NaN  ...         0     0
60522       60522      NaN  ...         0     0
60523       60523      NaN  ...         0     0

[60524 rows x 13 columns]>

Definição de key-words

Key-words (inglês)

In [ ]:
key_words_english = ['flood',
             'flooded',
             'it’s flooding',
             'to flood',
             'rain',
             'had been rained',
             'rainy',
             'heavy rain',
             'drizzle',
             'it’s raining',
             'storm',
             'storms'
            ]

Key-words (alemão)

In [ ]:
key_words_germany = ['hochwasser',
             'duden',
             'flut',
             'überschwemmung',
             'deich',
             'sandsack'
            ]

join key-words

In [ ]:
key_words = key_words_english + key_words_germany

Filtro por palavras-chave

In [ ]:
def select_tweets_with_key_words (df):
  # retorna os tweets que contem as palavras-chave
  return df[df.tweet.str.contains('|'.join(key_words), na=False)]

Verificação de quantidade tweets contendo palavras-chave

In [ ]:
tweets_with_key_words = select_tweets_with_key_words(df)
len(tweets_with_key_words)

773

## Aplicação da técnica

Calcular frequência absoluta

In [ ]:
def calcule_absolute_frequency (tweets):
  '''
  define um dicionário para registrar as frequências absolutas
  onde criamos um dicionário global do dataframe e registramos
  a frequência das palavras-chave 
  '''
  absolute_frequency = {}
  
  for word in key_words:
    sum_word = 0
    for tweet in tweets:
      if (tweet.count(word) > 0): 
        sum_word += 1
    absolute_frequency[word] = sum_word
  
  return absolute_frequency

Calcular frequência relativa

In [ ]:
def calcule_relative_frequency (tweet, word, absolute_frequency):
  '''
  a frequência relativa é a quantidade de aparições de palavras-chave dividio
  pela frequência absoluta dessa palavra-chave
  '''
  return tweet.count(word)/absolute_frequency

Calcular entropia

In [ ]:
import math

def calcule_tweets_entropy (data):
  # definimos um dicionári para a entropia dos tweets
  entropies = {}
  # realizamos a construção do dicionário de frequência absoluta das palavras chave no tweet
  absolute_frequency = calcule_absolute_frequency(data.tweet)

  for id, tweet in enumerate(data.tweet):
    frequences_array = []
    for word in key_words:
      # se houver pelo menos uma palavra-chave no tweet
      if (tweet.count(word) > 0):
        # calcule a frequencia relativa dela no tweet
        relative_frequency = calcule_relative_frequency(tweet, word, absolute_frequency[word])
        # adiciona o valor da frequência relativa na lista
        frequences_array.append(relative_frequency * math.log2(1/relative_frequency))
    # adiciona no dicionário os valores das entropias vinculados a seu respectivo id
    entropies[str(data.iloc[id, 0])] = sum(frequences_array)
  return entropies 

**Main**

In [ ]:
result = calcule_tweets_entropy(tweets_with_key_words)

## Válidação

quantos é maior que 0.5? E quais as situações desses dados?

In [ ]:
for item in result:
  if result.get(item) > 0.5:
    print(f'{item}: {result[item]}')
    print(df.iloc[int(item)].tweet, df.iloc[int(item)].flood)

9563: 0.6761069445245729
the river elbe flooded the town of desden before we arrived. this town and prague were under state of… http://t.co/ig2xhfwtsx 1
19731: 0.5526983718038826
heavy rains at fra as colleagues took off to sfo for #netapp foresight 2013, myself headed to bos for @redhatsummit 0
19801: 0.8431485898729221
waking up in the morning to heavy rain again... #rain #rainyweather #heavyrain #water #waterfalling… http://t.co/9tfizsqooh 0
31303: 0.5526983718038826
just changed the car due heavy rain and terrible weather audi a8l w12 http://t.co/qh6xzupbe2 0
35741: 0.6761069445245729
@josephmorgan need your help:/ our house was flooded http://t.co/whesrsgoe7 1
53967: 0.6761069445245729
germany is half flooded #naturaldisaster 1
59401: 0.6761069445245729
@josephmorgan need your help:/ our house was flooded http://t.co/0qrwxofdhi 1


Analisando correlação entre a distância geográfica da área de inundação e entropia

Para calcular essa correlação utilizamos o coeficiente de correlação de Person